In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理
def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)
    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")
    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）
def tokenizer_punctuation(text):
    return text.strip().split()

# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cats+'))

区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats']


In [3]:
import torchtext
from torchtext.data.utils import get_tokenizer

MAX_LENGTH = 30

#テキストに処理を行うFieldを定義
#fix_lengthはtokenの数
SRC = torchtext.data.Field(sequential=True, use_vocab=True, tokenize=tokenizer_with_preprocessing,
                            lower=True, include_lengths=True, batch_first=True, fix_length=MAX_LENGTH)

TRG = torchtext.data.Field(sequential=True, use_vocab=True, tokenize=tokenizer_with_preprocessing,
                            lower=True, include_lengths=True, batch_first=True, fix_length=MAX_LENGTH,
                            init_token='<cls>', eos_token='<eos>')

#pandasでcsvを保存するときに、labelをintでキャストしておかないとエラーでるから注意
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='drive/My Drive/dataset/DailyDialog/', train='train.csv', validation='validation.csv', 
    test='test.csv', format='csv', fields=[('src', SRC), ('trg', TRG)])

In [4]:
SRC.build_vocab(train_ds)
TRG.build_vocab(train_ds)
print(TRG.vocab.stoi)
print(len(TRG.vocab.stoi))

defaultdict(<function _default_unk_index at 0x7fd04b7aa620>, {'<unk>': 0, '<pad>': 1, '<cls>': 2, '<eos>': 3, '.': 4, 'i': 5, ',': 6, 'you': 7, 'the': 8, 'to': 9, 'a': 10, 'it': 11, 's': 12, 'that': 13, 'is': 14, 'and': 15, 't': 16, 'have': 17, '’': 18, 'of': 19, 'in': 20, 'for': 21, 'do': 22, 'what': 23, 'can': 24, 'we': 25, 'are': 26, 'your': 27, 'my': 28, 'me': 29, 'yes': 30, 'be': 31, 'm': 32, 'like': 33, 'but': 34, 'on': 35, 'this': 36, 'how': 37, 'll': 38, 'don': 39, 'with': 40, 'so': 41, 'about': 42, 'will': 43, 'no': 44, 'there': 45, 'well': 46, 'not': 47, 'at': 48, 'think': 49, 'know': 50, 'just': 51, 'would': 52, 'good': 53, 'all': 54, 'here': 55, 'get': 56, 'he': 57, 'right': 58, 'one': 59, 'go': 60, 'oh': 61, 'they': 62, 'really': 63, 'was': 64, 'see': 65, 'very': 66, 'if': 67, 'want': 68, 'please': 69, 'time': 70, 'some': 71, 'much': 72, 'too': 73, 'sure': 74, 'ok': 75, 'take': 76, 'let': 77, 'am': 78, 're': 79, 'now': 80, 've': 81, 'thank': 82, 'need': 83, 'going': 84, 'a

In [5]:
from torchtext import data

train_dl = data.Iterator(train_ds, batch_size=64, train=True)
val_dl = data.Iterator(val_ds, batch_size=64, train=False, sort=False)
batch = next(iter(val_dl))
print(batch.src[0].shape)
print(batch.trg[0].shape)

torch.Size([64, 30])
torch.Size([64, 30])


In [6]:
"""
#Tutorial
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), batch_first=True, bidirectional=True)

    def forward(self, input_seq, hidden=None):
        embedded = self.embedding(input_seq) #[64, 30, hidden]
        outputs, hidden = self.gru(embedded, hidden) #[64, 30, 1000], [4, 64, 500]
        # 前方向の隠れ状態と、後ろ方向の隠れ状態を足している
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden
"""

'\n#Tutorial\nclass EncoderRNN(nn.Module):\n    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):\n        super(EncoderRNN, self).__init__()\n        self.n_layers = n_layers\n        self.hidden_size = hidden_size\n        self.embedding = embedding\n        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,\n                          dropout=(0 if n_layers == 1 else dropout), batch_first=True, bidirectional=True)\n\n    def forward(self, input_seq, hidden=None):\n        embedded = self.embedding(input_seq) #[64, 30, hidden]\n        outputs, hidden = self.gru(embedded, hidden) #[64, 30, 1000], [4, 64, 500]\n        # 前方向の隠れ状態と、後ろ方向の隠れ状態を足している\n        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]\n        # Return output and final hidden state\n        return outputs, hidden\n'

In [7]:
class EncoderRNN(nn.Module):
  def __init__(self, hidden_size, vocab_size, dropout=0):
    super(EncoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(vocab_size, hidden_size)
    self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
    self.thought = nn.Linear(hidden_size*2, hidden_size)


  def forward(self, input_seq, hidden=None):
    embedded = self.embedding(input_seq) #[64, 30, 600]
    outputs, (hn, cn) = self.lstm(embedded) #[64, 30, 1200], ([2, 64, 600], [2, 64, 600])
    outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] #[64, 30, 600]
    thought_vector = torch.cat((hn[0], cn[0]), -1) #[64, 1200]
    thought_vector = self.thought(thought_vector).unsqueeze(0) #[1, 64, 600]

    return outputs, thought_vector

In [8]:
#エンコーダテスト
hidden_size = 600
dropout = 0.1
batch_size = 64

batch = next(iter(val_dl))
print(batch.src[0].shape)

encoder = EncoderRNN(hidden_size, len(SRC.vocab.stoi), dropout)
encoder_outputs, thought_vector = encoder(batch.src[0])
print(thought_vector.shape)

torch.Size([64, 30])
torch.Size([1, 64, 600])


In [9]:
"""
#Tutorial
class LuongAttnDecoderRNN(nn.Module):
  def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
    super(LuongAttnDecoderRNN, self).__init__()
    self.attn_model = attn_model
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    self.dropout = dropout

    self.embedding = embedding
    self.embedding_dropout = nn.Dropout(dropout)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout), batch_first=True)
    self.score = nn.Linear(hidden_size, hidden_size)
    self.concat = nn.Linear(hidden_size * 2, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    
  def forward(self, input_step, last_hidden, encoder_outputs):
    embedded = self.embedding(input_step)
    embedded = self.embedding_dropout(embedded)
    embedded = embedded.unsqueeze(1)
        
    rnn_output, hidden = self.gru(embedded, last_hidden) #[64, 1, 500] [2, 64, 500]
    print('encoder_outputs', encoder_outputs.shape)
    energy = self.score(encoder_outputs) # [64, 30, 500]
    print('energy', energy.shape)
    attn_weights = torch.sum(rnn_output*energy, dim=2) #[64, 30]
    print('attn_weight', attn_weights.shape)
    attn_weights = F.softmax(attn_weights, dim=1).unsqueeze(1) # [64, 1, 30]

    context = attn_weights.bmm(encoder_outputs) #[64, 1, 500]
    print('context', context.shape)
    rnn_output = rnn_output.squeeze(1) #[64, 500]
    context = context.squeeze(1) #[64, 500]
    concat_input = torch.cat((rnn_output, context), 1) #[64, 1000]
    concat_output = torch.tanh(self.concat(concat_input))
    output = self.out(concat_output)
    output = F.softmax(output, dim=1)

    return output, hidden
"""

"\n#Tutorial\nclass LuongAttnDecoderRNN(nn.Module):\n  def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):\n    super(LuongAttnDecoderRNN, self).__init__()\n    self.attn_model = attn_model\n    self.hidden_size = hidden_size\n    self.output_size = output_size\n    self.n_layers = n_layers\n    self.dropout = dropout\n\n    self.embedding = embedding\n    self.embedding_dropout = nn.Dropout(dropout)\n    self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout), batch_first=True)\n    self.score = nn.Linear(hidden_size, hidden_size)\n    self.concat = nn.Linear(hidden_size * 2, hidden_size)\n    self.out = nn.Linear(hidden_size, output_size)\n    \n  def forward(self, input_step, last_hidden, encoder_outputs):\n    embedded = self.embedding(input_step)\n    embedded = self.embedding_dropout(embedded)\n    embedded = embedded.unsqueeze(1)\n        \n    rnn_output, hidden = self.gru(embedded, last_hidden) #

In [10]:
class LuongAttnDecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size, dropout=0.1):
    super(LuongAttnDecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout = dropout

    self.embedding = nn.Embedding(output_size, hidden_size)
    self.embedding_dropout = nn.Dropout(dropout)
    self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
    self.score = nn.Linear(hidden_size, hidden_size)
    self.concat = nn.Linear(hidden_size * 2, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    
  def forward(self, input_step, decoder_hidden, encoder_outputs):
    embedded = self.embedding(input_step)
    embedded = self.embedding_dropout(embedded)
    embedded = embedded.unsqueeze(1) #[64, 1, 600]
    
    #記憶セルはencoderから引っ張ってこない
    rnn_output, hidden = self.lstm(embedded, decoder_hidden) #[64, 1, 600] ([1, 64, 600], [1, 64, 600])
    energy = self.score(encoder_outputs) # [64, 30, 600]
    attn_weights = torch.sum(rnn_output*energy, dim=2) #[64, 30]
    attn_weights = F.softmax(attn_weights, dim=1).unsqueeze(1) # [64, 1, 30]

    context = attn_weights.bmm(encoder_outputs) #[64, 1, 500]
    rnn_output = rnn_output.squeeze(1) #[64, 500]
    context = context.squeeze(1) #[64, 500]
    concat_input = torch.cat((rnn_output, context), 1) #[64, 1000]
    concat_output = torch.tanh(self.concat(concat_input))
    output = self.out(concat_output)
    output = F.softmax(output, dim=1)

    return output, hidden

In [17]:
def binaryMatrix(l, value=TRG.vocab.stoi['<pad>']):
    m = []
    for i, seq in enumerate(l):
      if seq == TRG.vocab.stoi['<pad>']:
        m.append(False)
      else:
        m.append(True)
    return m

def maskNLLLoss(inp, target):
    mask = target
    mask = binaryMatrix(mask)
    mask = torch.BoolTensor(mask)
    mask = mask.to(device)
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [12]:
decoder = LuongAttnDecoderRNN(hidden_size, len(TRG.vocab.stoi),  dropout)
decoder_input = torch.LongTensor([TRG.vocab.stoi['<cls>'] for _ in range(batch_size)])
embedding = nn.Embedding(len(SRC.vocab.stoi), hidden_size)
print(decoder_input.shape)

cn = torch.zeros(1, batch_size, hidden_size)
#cn = torch.zeros(1, batch_size, hidden_size, device=device)
decoder_hidden = (thought_vector, cn)
target_variable = batch.trg[0]


for t in range(30):
  decoder_output, decoder_hidden = decoder(
      decoder_input, decoder_hidden, encoder_outputs
  ) #[64, 単語種類数], [2, 64, 500]
  # Teacher forcing: next input is current target
  _, topi = decoder_output.topk(1)
  decoder_input = torch.LongTensor([topi[i] for i in range(batch_size)])
  #decoder_input = decoder_input.to(device)
  #loss += criterion(decoder_output, target_variable[:, t])
  mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[:, t])
  
decoder_output, decoder_hidden = decoder(
    decoder_input, decoder_hidden, encoder_outputs
) #[64, 単語種類数], [2, 64, 500]

print('decoder_output', decoder_output.shape)
print('decoder_hidden', decoder_hidden[0].shape)

torch.Size([64])
decoder_output torch.Size([64, 13046])
decoder_hidden torch.Size([1, 64, 600])


In [13]:
def a_batch_loss(input_variable, target_variable, max_target_len, encoder, decoder, 
                 encoder_optimizer, decoder_optimizer, criterion, phase):
  total_loss = 0 #1batchのloss
  # Zero gradients
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  n_totals = 0
  print_losses = []
  
  #エンコーダの出力
  encoder_outputs, thought_vector = encoder(input_variable)
  #['<cls>']を生成
  decoder_input = torch.LongTensor([TRG.vocab.stoi['<cls>'] for _ in range(batch_size)]) #[64]
  decoder_input = decoder_input.to(device)
  #エンコーダの最後の隠れ状態を使用、記憶セルは0を入力
  cn = torch.zeros(1, batch_size, hidden_size, device=device)
  decoder_hidden = (thought_vector, cn)

  #teaching_forceを使う
  loss = 0 #1batchの中の1センテンスのloss
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  if use_teacher_forcing:
    for t in range(max_target_len):
      decoder_output, decoder_hidden = decoder(
          decoder_input, decoder_hidden, encoder_outputs
      ) #[64, 単語種類数], [2, 64, 500]
      # Teacher forcing: next input is current target
      decoder_input = target_variable[:, t] #[64], teaching_forceの場合、正解データを次に入力する
      #loss += criterion(decoder_output, target_variable[:, t])
      mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[:, t])
      loss += mask_loss
      print_losses.append(mask_loss.item() * nTotal)
      n_totals += nTotal
    #total_loss += loss / max_target_len #1バッチ分のloss
    
  else:
    for t in range(max_target_len):
      decoder_output, decoder_hidden = decoder(
          decoder_input, decoder_hidden, encoder_outputs
      ) #[64, 単語種類数], [2, 64, 500]
      # Teacher forcing: next input is current target
      _, topi = decoder_output.topk(1)
      decoder_input = torch.LongTensor([topi[i] for i in range(batch_size)])
      decoder_input = decoder_input.to(device)
      #loss += criterion(decoder_output, target_variable[:, t])
      mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[:, t])
      loss += mask_loss
      print_losses.append(mask_loss.item() * nTotal)
      n_totals += nTotal
    #total_loss += (loss / max_target_len) #1バッチ分のloss
    
  if phase == 'train':
    loss.backward()
    #total_loss.backward()
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()
    decoder_optimizer.step()
  return sum(print_losses) / n_totals
  #return total_loss #1バッチ分のloss

In [14]:
import random

def train_model(dataloaders_dict, num_epochs, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    print("Training...")
    #エポック
    for epoch in range(num_epochs):
      for phase in ['train', 'val']:
        if phase == 'train':
          encoder.train()
          decoder.train()
        else:
          encoder.eval()
          decoder.eval()
        print_loss = 0 #1epochのloss

        for i, batch in enumerate(dataloaders_dict[phase]): 
          input_variable = batch.src[0].to(device) #(64, 30)
          target_variable = batch.trg[0].to(device) #(64, 30)
          max_target_len = max(batch.trg[1])
          if target_variable.shape[0] == 64:
            total_loss = a_batch_loss(input_variable, target_variable, max_target_len, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, phase) #1バッチ分のloss     
            print_loss += total_loss #1epochのlossをprint_lossに加えていく

        #損失をだす
        print("epoch: {}; phase: {}; Average loss: {:.4f}; PPL: {:.4f}".format(epoch+1, phase, print_loss/i, math.exp(print_loss/i) ))  

In [15]:
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

encoder = EncoderRNN(hidden_size, len(SRC.vocab.stoi), dropout)
decoder = LuongAttnDecoderRNN(hidden_size, len(TRG.vocab.stoi),  dropout)

encoder = encoder.to(device)
decoder = decoder.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=TRG.vocab.stoi['<pad>'])

Models built and ready to go!


In [18]:
from torch import optim

clip = 1.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
num_epochs = 3

dataloaders_dict = {"train": train_dl, "val": val_dl}

encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

encoder.train()
decoder.train()

Building optimizers ...
Starting Training!


In [19]:
train_model(dataloaders_dict, num_epochs, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

Training...
epoch: 1; phase: train; Average loss: 4.2681; PPL: 71.3830
epoch: 1; phase: val; Average loss: 3.8512; PPL: 47.0489
epoch: 2; phase: train; Average loss: 3.6794; PPL: 39.6211
epoch: 2; phase: val; Average loss: 3.6574; PPL: 38.7621
epoch: 3; phase: train; Average loss: 3.4579; PPL: 31.7493
epoch: 3; phase: val; Average loss: 3.5508; PPL: 34.8409


In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        encoder_outputs, encoder_hidden = self.encoder(input_seq)
        decoder_hidden = encoder_hidden[:decoder.n_layers] #最終層の隠れ状態を使う
        print(decoder_hidden.shape)
        decoder_input = torch.LongTensor([TRG.vocab.stoi['<cls>']]).to(device)
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            #decoder_input = torch.unsqueeze(decoder_input, 0)
            
        return all_tokens, all_scores

In [ ]:
import unicodedata

def indexesFromSentence(sentence):
    return [SRC.vocab.stoi[word] if word in SRC.vocab.stoi else SRC.vocab.stoi['<unk>'] for word in sentence.split(' ')]
    
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def evaluate(encoder, decoder, searcher, sentence, max_length=MAX_LENGTH):
    indexes_batch = indexesFromSentence(sentence)
    #lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    lengths = len(indexes_batch)
    input_batch = torch.LongTensor(indexes_batch).view(1, -1).to(device)
    #lengths = lengths.to(device)
    tokens, scores = searcher(input_batch, lengths, max_length)
    decoded_words = [TRG.vocab.itos[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher):
    input_sentence = ''
    while(1):
        try:
            input_sentence = input('> ').lower()
            if input_sentence == 'q' or input_sentence == 'quit': break
            #ノイズ処理
            input_sentence = normalizeString(input_sentence)
            output_words = evaluate(encoder, decoder, searcher, input_sentence)
            output_words[:] = [x for x in output_words if not (x == '<eos>' or x == '<pad>')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)

evaluateInput(encoder, decoder, searcher)

> where are you form ?
torch.Size([2, 1, 500])
Bot: <cls> i m not sure . it . it . it . it . it . it . it . it
> what do i have to do ?
torch.Size([2, 1, 500])
Bot: <cls> you can get a decision . it . it . it . it . it . it . it on the way
> today i have a breakfast. it tasted very good .
torch.Size([2, 1, 500])
Bot: <cls> i m glad you can get it . it . it . it . it . it . it . it .
> quit
